# Monte-Carlo to explore the optimal mask for QUBIC

In [ ]:
%matplotlib inline
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from importlib import reload

# Specific qubic modules
from pysimulators import FitsArray
import pysm
import qubic
from qubic import QubicSkySim as qss
from qubic import NamasterLib as nam

rc('figure', figsize=(12, 8))
rc('font', size=15)
rc('text', usetex=False)

In [ ]:
dictfilename = 'test_cross-spectra.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside']=256
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
print(center)

print(d['nf_sub'])

# Restore a QUBIC typical coverage
cov = hp.ud_grade(hp.read_map('sample_coverage_qubic.fits', verbose=False), d['nside'])
cov /= np.max(cov)
hp.mollview(cov)

Now we run a monte-carlo using two possibilities for the mask: flat or following coverage. We also explore various values for the coverage cut that defines the extension of the mask

In [ ]:
import pickle
reload(qss)
reload(nam)

# noise on maps (small here)
sigma_sec = 70

# Namaster objects ell-range
lmin = 20
lmax = 2 * d['nside'] - 1

##### MC Parameters
delta_ell_values = np.array([10., 20., 30., 40., 50])
covcut_values = np.array([0.01, 0.05, 0.1, 0.2, 0.5])
nbmc = 1000
# delta_ell_values = np.array([20., 30.])
# covcut_values = np.array([0.1, 0.05])
# nbmc = 5


for imc in range(nbmc):
    print('MC iteration {} over {}'.format(imc, nbmc))
    ### Create two fake QUBIC observations with same CMB and different noise
    ### At each MC step the CMB will different
    print(' - Generating CMB+Noise and Noise')
    seed = None
    sky_config = {'cmb': seed}
    Qubic_sky = qss.Qubic_sky(sky_config, d)

    ### Input Th Spectra
    input_cell = Qubic_sky.input_cmb_spectra
    ellth= np.arange(input_cell.shape[0])

    nmaps = 2
    all_maps = np.zeros((nmaps, 12*d['nside']**2, 3))
    all_maps_noise = np.zeros((nmaps, 12*d['nside']**2, 3))
#     for s in range(4):
#         subplot(2,2,s+1)
#         plot(ellth[:512], input_cell[:512,s],'k')
        
    ### Noisy maps with CMB
    for i in range(nmaps):
        all_maps[i,:,:] = Qubic_sky.get_partial_sky_maps_withnoise(cov, sigma_sec=sigma_sec)
        all_maps_noise[i,:,:] = Qubic_sky.create_noise_maps(sigma_sec, cov)
        
    ### Now loop on possible analyses parameters
    for icc in range(len(covcut_values)):
        print(' - Doing covcut = {}'.format(covcut_values[icc]))
        okpix = cov > np.max(cov) * covcut_values[icc]
        #print(' - We have {} pixels inside'.format(okpix.sum()))

        ######## MASKS ######################
        ### First case: Flat weighting
        maskpix_flat = np.zeros(12*d['nside']**2)
        maskpix_flat[okpix] = 1
        ### First case: Flat weighting
        maskpix_cov = np.zeros(12*d['nside']**2)
        maskpix_cov[okpix] = cov[okpix]
        ######################################
        
        
        ### Now loop on delta_ell
        for idl in range(len(delta_ell_values)):
            w = None
            print('    - Doing DeltaEll = {}'.format(delta_ell_values[idl]))
            ### Falt
            Namaster_flat = nam.Namaster(maskpix_flat, lmin=lmin, lmax=lmax, delta_ell=delta_ell_values[idl])
            ### Cov
            Namaster_cov = nam.Namaster(maskpix_cov, lmin=lmin, lmax=lmax, delta_ell=delta_ell_values[idl])
            
            ### Cross-Cls flat for CMB + Noise
            #print('         * CMB+Noise - Flat weighting')
            leff, cls_flat, w = Namaster_flat.get_spectra(all_maps[0,:,:].T, 
                                                    map2=all_maps[1,:,:].T,
                                                    purify_e=False, purify_b=True,
                                                    verbose=False,
                                                    beam_correction=Qubic_sky.instrument['beams'],
                                                    w=None)

            ### Cross-Cls cov for CMB + Noise
            #print('         * CMB+Noise - Cov weighting')
            leff, cls_cov, w = Namaster_cov.get_spectra(all_maps[0,:,:].T, 
                                                    map2=all_maps[1,:,:].T,
                                                    purify_e=False, purify_b=True,
                                                    verbose=False,
                                                    beam_correction=Qubic_sky.instrument['beams'],
                                                    w=w)

            ### Cross-Cls flat for CMB + Noise
            #print('         * Noise Only - Flat weighting')
            leff, cls_flat_noise, w = Namaster_flat.get_spectra(all_maps_noise[0,:,:].T, 
                                                            map2=all_maps_noise[1,:,:].T,
                                                            purify_e=False, purify_b=True,
                                                            verbose=False,
                                                            beam_correction=Qubic_sky.instrument['beams'],
                                                            w=w)

            ### Cross-Cls cov for CMB + Noise
            #print('         * Noise Only - Cov weighting')
            leff, cls_cov_noise, wcn = Namaster_cov.get_spectra(all_maps_noise[0,:,:].T, 
                                                        map2=all_maps_noise[1,:,:].T,
                                                        purify_e=False, purify_b=True,
                                                        verbose=False,
                                                        beam_correction=Qubic_sky.instrument['beams'],
                                                        w=w)
            
#             for s in range(4):
#                 subplot(2,2,s+1)
#                 plot(leff, cls_flat[:,s],label='Cl Flat')
#                 plot(leff, cls_cov[:,s],label='Cl Cov')
#                 plot(leff, cls_flat_noise[:,s],label='Cl Noise Flat')
#                 plot(leff, cls_cov_noise[:,s],label='Cl Noise Cov')
#                 legend(fontsize=8)
            
            
        
            ### Write files to disk
            rnddata = qss.random_string(10)
            rndnoise = qss.random_string(10)
            directory = '/Users/hamilton/Qubic/QubicGeneralPaper2020/Sims/Sims_Optimize_Weighting/Noise_70/'

            with open(directory+'cls_flat_data_covcut_{}_deltaell_{}_{}.pickle'.format(covcut_values[icc], delta_ell_values[idl], rnddata), 'wb') as handle:
                pickle.dump([leff, cls_flat], handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(directory+'cls_cov_data_covcut_{}_deltaell_{}_{}.pickle'.format(covcut_values[icc], delta_ell_values[idl], rnddata), 'wb') as handle:
                pickle.dump([leff, cls_cov], handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(directory+'cls_flat_noise_covcut_{}_deltaell_{}_{}.pickle'.format(covcut_values[icc], delta_ell_values[idl], rnddata), 'wb') as handle:
                pickle.dump([leff, cls_flat_noise], handle, protocol=pickle.HIGHEST_PROTOCOL)

            with open(directory+'cls_cov_noise_covcut_{}_deltaell_{}_{}.pickle'.format(covcut_values[icc], delta_ell_values[idl], rnddata), 'wb') as handle:
                pickle.dump([leff, cls_cov_noise], handle, protocol=pickle.HIGHEST_PROTOCOL)


        



Analyze the MC in notebook: Optimize-Mask-QUBIC-AnalyzeMC.Rmd